In [1]:
from __future__ import print_function

import keras
from keras.datasets import mnist
from keras.models import Model
from keras.layers import Input, Dense, TimeDistributed
from keras.layers import LSTM

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [3]:
ver=pd.read_csv("venezia_extended.csv")

In [4]:
print(ver.head(10))

   Unnamed: 0             datetime     datetimetuple  hour_inday  level  \
0           0  1983-01-01 01:00:00   (1983, 1, 1, 1)           1   44.0   
1           1  1983-01-01 02:00:00   (1983, 1, 1, 2)           2   35.0   
2           2  1983-01-01 03:00:00   (1983, 1, 1, 3)           3   23.0   
3           3  1983-01-01 04:00:00   (1983, 1, 1, 4)           4   10.0   
4           4  1983-01-01 05:00:00   (1983, 1, 1, 5)           5    1.0   
5           5  1983-01-01 06:00:00   (1983, 1, 1, 6)           6   -1.0   
6           6  1983-01-01 07:00:00   (1983, 1, 1, 7)           7    6.0   
7           7  1983-01-01 08:00:00   (1983, 1, 1, 8)           8   17.0   
8           8  1983-01-01 09:00:00   (1983, 1, 1, 9)           9   31.0   
9           9  1983-01-01 10:00:00  (1983, 1, 1, 10)          10   41.0   

   level_normalized  lunar_daynum  lunarday   lunardaytuple  lunarmonth  \
0          0.621919             1        18  (1982, 11, 18)          11   
1          0.312960     

In [ ]:
#yearperi_train=yearperi[189272:259272]
#yearperi_test=yearperi[259272:]

In [9]:
lunaryearlist=list(ver["lunaryear"])
lunarmonthlist=list(ver["lunarmonth"])
lunardaylist=list(ver["lunarday"])
hrlist=list(ver["hour_inday"])
waterlist=list(ver["level_normalized"])

In [43]:
print(len(waterlist))
print(len(lunardaylist))

289272
289272


In [44]:
print(waterlist[289271])

-0.2019715810710306


In [70]:
rectrain=[]

leveltrain=[]
for i in range(189272,259272):
    slice_=ver[(ver["lunaryear"]==lunaryearlist[i]-3) & (ver["lunarmonth"]==lunarmonthlist[i])&(ver["lunarday"]==lunardaylist[i])&(ver["hour_inday"]==hrlist[i]) ]
    if(slice_.shape[0]!=0):
        startpoint=slice_.iloc[0][0]
        ts=[]
        for j in range(-24*3+1,1):
            ts.append(waterlist[startpoint+(15*24*j)])
        rectrain.append(ts)
        leveltrain.append(waterlist[i])

In [71]:
rectest=[]
leveltest=[]
for i in range(259272,len(waterlist)):
    slice_=ver[(ver["lunaryear"]==lunaryearlist[i]-3) & (ver["lunarmonth"]==lunarmonthlist[i])&(ver["lunarday"]==lunardaylist[i])&(ver["hour_inday"]==hrlist[i]) ]
    if(slice_.shape[0]!=0):
        startpoint=int(slice_.iloc[0][0])
        ts=[]
        for j in range(-71,1):
            ts.append(waterlist[startpoint+360*j])
            #if(i==len(waterlist)-1):
                #print(startpoint+360*j)
                #print(waterlist[startpoint-360*j])
            #print(j)
        #if(i==len(waterlist)-1):
         #   print(slice_)
          #  print(startpoint)
           # print(waterlist[startpoint-360])
        rectest.append(ts)
        leveltest.append(waterlist[i])

In [72]:
#print(len(rectrain))
print(len(rectest))

29830


In [73]:
print(waterlist[262679])

0.07265860688562839


In [74]:
rectrain=np.array(rectrain)
rectest=np.array(rectest)
leveltrain=np.array(leveltrain)
leveltest=np.array(leveltest)

In [76]:
print(rectest[29829])
print(leveltest[29829])

[[ 1.34282323  2.0293987   1.78909728]
 [ 0.96520672  1.03386426  1.377152  ]
 [ 1.06819304  1.48013832  0.99953549]
 [ 0.58759021  0.31296002  0.07265861]
 [-0.16764281 -0.40794422 -0.37361545]
 [-0.13331403 -0.06465649  0.14131615]
 [-0.27062913  0.27863125 -0.3049579 ]
 [ 0.17564493  0.10698738  0.45027512]
 [ 0.62191898  0.03832983  0.55326144]
 [-0.06465649 -0.16764281 -0.09898526]
 [ 0.31296002 -0.442273    0.17564493]
 [-0.33928668 -0.64824564  0.03832983]
 [ 0.03832983 -0.16764281  0.45027512]
 [ 0.55326144  0.45027512  0.14131615]
 [ 1.75476851  1.03386426  0.89654917]
 [ 1.48013832  1.34282323  0.99953549]
 [ 1.03386426  1.72043973  0.58759021]
 [ 0.45027512  0.55326144  1.96074115]
 [ 1.61745341  1.44580955  1.23983691]
 [ 1.34282323  1.51446709  1.44580955]
 [ 1.72043973  1.27416568  0.89654917]
 [ 0.65624776  1.06819304  0.8622204 ]
 [ 0.17564493  2.09805624  0.24430247]
 [ 0.48460389  0.07265861  0.45027512]]
-0.2019715810710306


In [ ]:
rectrain=rectrain.reshape(rectrain.shape[0],72,1)
rectest=rectest.reshape(rectest.shape[0],72,1)
row,col=rectrain.shape[1:]

epoch=25

row_hidden=64
col_hidden=64

x=Input(shape=(row,col))

#encoded rows
#encoded_rows = TimeDistributed(LSTM(row_hidden))(x)

#encoded cols
#encoded_cols= LSTM(col_hidden)(encoded_rows)
encoded_cols= LSTM(col_hidden)(x)

#最后的全连接
fin= Dense(10, activation='tanh')(encoded_cols)
#prediction= Dense(1, activation='tanh')(encoded_cols)

#
prediction=Dense(1,)(fin)

#summarizing and setting model
model = Model(x, prediction)
#这个model弄的很有趣，直接包含中间了的么
model.compile(loss='MSE',
              optimizer='adamax')

model.summary()

FHistory=model.fit(rectrain, leveltrain,
          #batch_size=32,
          epochs=25,
          verbose=1,
          validation_data=(rectest, leveltest))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        (None, 72, 1)             0         
_________________________________________________________________
lstm_10 (LSTM)               (None, 64)                16896     
_________________________________________________________________
dense_19 (Dense)             (None, 10)                650       
_________________________________________________________________
dense_20 (Dense)             (None, 1)                 11        
Total params: 17,557
Trainable params: 17,557
Non-trainable params: 0
_________________________________________________________________
Train on 69566 samples, validate on 29830 samples
Epoch 1/25
69566/69566 [==============================] - 81s 1ms/step - loss: 0.3742 - val_loss: 0.3577
Epoch 2/25
69566/69566 [==============================] - 80s 1ms/step - loss: 0.3255 - val_loss: 0.3210
Epoch 3/25